## Establish a connection to the XRP Publicy Ledger

## Manage keys and wallets
## `xrpl.wallet`
* Use the xrpl.wallet module to create a wallet from a given seed or or via a Testnet faucet.
* To create a wallet from a seed (in this case, the value generated using xrpl.keypairs):

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from xrpl.clients import JsonRpcClient
JSON_RPC_URL = "https://s.altnet.rippletest.net:51234"
client = JsonRpcClient(JSON_RPC_URL)

In [3]:
# Create a wallet using the testnet faucet:
# https://xrpl.org/xrp-testnet-faucet.html
from xrpl.wallet import generate_faucet_wallet
test_wallet = generate_faucet_wallet(client, debug=True)

Attempting to fund address rh8PexXJGyNMqYyKWMUvdZVAzzVXRUdQya
Faucet fund successful.


## To create a wallet from a Testnet faucet:

In [4]:
test_wallet = generate_faucet_wallet(client)
test_account = test_wallet.classic_address
print("Classic address:", test_account)

Classic address: rLkveviqgrvm6BFYa1MTjj6Y74W1rgn8Fz


## xrpl.core.keypairs
* Use the xrpl.core.keypairs module to generate seeds and derive keypairs and addresses from those seed values.

In [5]:
from xrpl.core import keypairs
seed = keypairs.generate_seed()
public, private = keypairs.derive_keypair(seed)
test_account = keypairs.derive_classic_address(public)
print("Here's the public key:")
print(public)
print("Here's the private key:")
print(private)
print("Store this in a secure place!")

Here's the public key:
ED229A6CA62892A59142A1A18FDA209807EF0D59CD270816832955B2A66633EAC5
Here's the private key:
ED87A2FB4A51E2758D382AE4D65236BB71B047F5B1894ADDE10B72A9E46299B526
Store this in a secure place!


In [10]:
# Prepare payment
from xrpl.models.transactions import Payment
from xrpl.utils import xrp_to_drops
my_tx_payment = Payment(
    account=test_account,
    amount=xrp_to_drops(22),
    destination="rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe",
)

In [11]:
# Sign the transaction
from xrpl.transaction import safe_sign_and_autofill_transaction

my_tx_payment_signed = safe_sign_and_autofill_transaction(my_tx_payment, test_wallet, client)

In [12]:
# Submit and send the transaction
from xrpl.transaction import send_reliable_submission

tx_response = send_reliable_submission(my_tx_payment_signed, client)

In [13]:
# Derive an x-address from the classic address:
# https://xrpaddress.info/
from xrpl.core import addresscodec
test_xaddress = addresscodec.classic_address_to_xaddress(test_account, tag=12345, is_test_network=True)
print("\nClassic address:\n\n", test_account)
print("X-address:\n\n", test_xaddress)


Classic address:

 r94k8wGrfQGzWSMSZEgkUpFMcCgN7UVJUo
X-address:

 T7i4ansso1BZNBQnPifnnYkJ6t25gipGgxz5EGZUy8jwNnt


# Serialize and sign transactions

* First serialize data from JSON and other formats into the XRP Ledger's canonical format
* Then to authorize the transaction by digitally signing it with the account's private key. The xrpl-py library provides several methods to simplify this process.

In [6]:
from xrpl.models.transactions import Payment
from xrpl.transaction import safe_sign_transaction, send_reliable_submission
from xrpl.ledger import get_latest_validated_ledger_sequence
from xrpl.account import get_next_valid_seq_number

current_validated_ledger = get_latest_validated_ledger_sequence(client)
test_wallet.sequence = get_next_valid_seq_number(test_wallet.classic_address, client)

# prepare the transaction
# the amount is expressed in drops, not XRP
# see https://xrpl.org/basic-data-types.html#specifying-currency-amounts
my_tx_payment = Payment(
    account=test_wallet.classic_address,
    amount="2200000",
    destination="rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe",
    last_ledger_sequence=current_validated_ledger + 20,
    sequence=test_wallet.sequence,
    fee="10",
)
# sign the transaction
my_tx_payment_signed = safe_sign_transaction(my_tx_payment,test_wallet)

# submit the transaction
tx_response = send_reliable_submission(my_tx_payment_signed, client)

# Get fee from the XRP Ledger


In [7]:
# use the get_fee function
from xrpl.ledger import get_fee
fee = get_fee(client)
print(fee)
# 10

10


## Payments on the XRP Ledger

In [8]:
from xrpl.models.transactions import Payment
from xrpl.transaction import send_reliable_submission, safe_sign_and_autofill_transaction
# prepare the transaction
# the amount is expressed in drops, not XRP
# see https://xrpl.org/basic-data-types.html#specifying-currency-amounts
my_tx_payment = Payment(
    account=test_wallet.classic_address,
    amount="2200000",
    destination="rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe"
)

# sign the transaction with the autofill method
# (this will auto-populate the fee, sequence, and last_ledger_sequence)
my_tx_payment_signed = safe_sign_and_autofill_transaction(my_tx_payment, test_wallet, client)
print(my_tx_payment_signed)
# Payment(
#     account='rMPUKmzmDWEX1tQhzQ8oGFNfAEhnWNFwz',
#     transaction_type=<TransactionType.PAYMENT: 'Payment'>,
#     fee='10',
#     sequence=16034065,
#     account_txn_id=None,
#     flags=0,
#     last_ledger_sequence=10268600,
#     memos=None,
#     signers=None,
#     source_tag=None,
#     signing_pub_key='EDD9540FA398915F0BCBD6E65579C03BE5424836CB68B7EB1D6573F2382156B444',
#     txn_signature='938FB22AE7FE76CF26FD11F8F97668E175DFAABD2977BCA397233117E7E1C4A1E39681091CC4D6DF21403682803AB54CC21DC4FA2F6848811DEE10FFEF74D809',
#     amount='2200000',
#     destination='rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe',
#     destination_tag=None,
#     invoice_id=None,
#     paths=None,
#    send_max=None,
#    deliver_min=None
# )

# submit the transaction
tx_response = send_reliable_submission(my_tx_payment_signed, client)

Payment(account='rLkveviqgrvm6BFYa1MTjj6Y74W1rgn8Fz', transaction_type=<TransactionType.PAYMENT: 'Payment'>, fee='10', sequence=27025239, account_txn_id=None, flags=0, last_ledger_sequence=27025266, memos=None, signers=None, source_tag=None, signing_pub_key='EDE7341AA658100BA4C62B3F5EF6B138B814F46F029EFD023B40DED45274899CB1', txn_signature='D39665230F991788B77D7FE0ACAD27A583051A20412E6216CF38355C154BEC228F49D56A788ED7E647E95A54C508BFA0EAB09A70EA8F2EC78C3566FD91E7C504', amount='2200000', destination='rPT1Sjq2YGrBMTttX4GZHjKu9dyfzbpAYe', destination_tag=None, invoice_id=None, paths=None, send_max=None, deliver_min=None)


# Subscribe to XRP ledger updates


In [ ]:
# Send subscribe and unsubscribe requests using the WebSocket Network Client.
from xrpl.clients import WebsocketClient
url = "wss://s.altnet.rippletest.net/"
from xrpl.models.requests import Subscribe, StreamParameter
req = Subscribe(streams=[StreamParameter.LEDGER])
# NOTE: this code will run forever without a timeout, until the process is killed
with WebsocketClient(url) as client:
    client.send(req)
    for message in client:
        print(message)
# {'result': {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': '7CD50477F23FF158B430772D8E82A961376A7B40E13C695AA849811EDF66C5C0', 'ledger_index': 18183504, 'ledger_time': 676412962, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'validated_ledgers': '17469391-18183504'}, 'status': 'success', 'type': 'response'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'BAA743DABD168BD434804416C8087B7BDEF7E6D7EAD412B9102281DD83B10D00', 'ledger_index': 18183505, 'ledger_time': 676412970, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183505'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'D8227DAF8F745AE3F907B251D40B4081E019D013ABC23B68C0B1431DBADA1A46', 'ledger_index': 18183506, 'ledger_time': 676412971, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183506'}
# {'fee_base': 10, 'fee_ref': 10, 'ledger_hash': 'CFC412B6DDB9A402662832A781C23F0F2E842EAE6CFC539FEEB287318092C0DE', 'ledger_index': 18183507, 'ledger_time': 676412972, 'reserve_base': 20000000, 'reserve_inc': 5000000, 'txn_count': 0, 'type': 'ledgerClosed', 'validated_ledgers': '17469391-18183507'}

## Combinations
#### Using these building blocks, I have created a simple Python app that can:
* Generate a wallet on the Testnet.
* Connects to the XRP Ledger.
* Look up and print information about the account I created.

In [9]:
# Define the network client
from xrpl.clients import JsonRpcClient
JSON_RPC_URL = "https://s.altnet.rippletest.net:51234/"
client = JsonRpcClient(JSON_RPC_URL)


# Create a wallet using the testnet faucet:
# https://xrpl.org/xrp-testnet-faucet.html
from xrpl.wallet import generate_faucet_wallet
test_wallet = generate_faucet_wallet(client, debug=True)

# Create an account str from the wallet
test_account = test_wallet.classic_address

# Derive an x-address from the classic address:
# https://xrpaddress.info/
from xrpl.core import addresscodec
test_xaddress = addresscodec.classic_address_to_xaddress(test_account, tag=12345, is_test_network=True)
print("\nClassic address:\n\n", test_account)
print("X-address:\n\n", test_xaddress)


# Look up info about your account
from xrpl.models.requests.account_info import AccountInfo
acct_info = AccountInfo(
    account=test_account,
    ledger_index="validated",
    strict=True,
)
response = client.request(acct_info)
result = response.result
print("response.status: ", response.status)
import json
print(json.dumps(response.result, indent=4, sort_keys=True))

Attempting to fund address r94k8wGrfQGzWSMSZEgkUpFMcCgN7UVJUo
Faucet fund successful.

Classic address:

 r94k8wGrfQGzWSMSZEgkUpFMcCgN7UVJUo
X-address:

 T7i4ansso1BZNBQnPifnnYkJ6t25gipGgxz5EGZUy8jwNnt
response.status:  ResponseStatus.SUCCESS
{
    "account_data": {
        "Account": "r94k8wGrfQGzWSMSZEgkUpFMcCgN7UVJUo",
        "Balance": "1000000000",
        "Flags": 0,
        "LedgerEntryType": "AccountRoot",
        "OwnerCount": 0,
        "PreviousTxnID": "731EF7242A4A609895FD080AC962C79A2C1DDF0C9B3000CBECFF9ADC9E909D75",
        "PreviousTxnLgrSeq": 27025250,
        "Sequence": 27025250,
        "index": "1F778C8A9644A7A1CC365752093C6932EBAAD80B0FAD0BC0C2272F6BE19F3AAD"
    },
    "ledger_hash": "55D81C2274D0FDCC781FB1B3B544187B6803CDA5E2EC46DECE4219994D4E129C",
    "ledger_index": 27025250,
    "validated": true
}
